In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
data = pd.read_csv('data/yama_pitching_data').dropna()

In [112]:
data

,game_pk,game_date,at_bat_number,pitch_number,pitch_type,batter_name,batter_id,p_hand,b_hand,same_hand_matchup,...,on_3b,on_2b,on_1b,inning,top_bot,p_pitch_count,pitcher_team,opponent,prev_pitch_type,prev_pitch_result
1,776185,2025-09-25,57,5,SI,Blaze Alexander,677942,R,R,1,...,0,0,1,6,0,2,8,0,SL,hit_into_play
2,776185,2025-09-25,57,4,FF,Blaze Alexander,677942,R,R,1,...,0,0,1,6,0,3,8,0,SI,ball
3,776185,2025-09-25,57,3,FF,Blaze Alexander,677942,R,R,1,...,0,0,1,6,0,4,8,0,FF,foul
4,776185,2025-09-25,57,2,FS,Blaze Alexander,677942,R,R,1,...,0,0,1,6,0,5,8,0,FF,ball
5,776185,2025-09-25,57,1,SI,Blaze Alexander,677942,R,R,1,...,0,0,1,6,0,6,8,0,FS,ball
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,778563,2025-03-18,5,1,FF,Seiya Suzuki,673548,R,R,1,...,0,0,1,1,0,67,0,0,FS,ball
2787,778563,2025-03-18,4,5,FF,Ian Happ,664023,R,L,0,...,0,0,0,1,0,69,0,0,FF,ball
2788,778563,2025-03-18,4,4,FF,Ian Happ,664023,R,L,0,...,0,0,0,1,0,70,0,0,FF,called_strike
2789,778563,2025-03-18,4,3,FF,Ian Happ,664023,R,L,0,...,0,0,0,1,0,71,0,0,FF,called_strike


In [2]:
data.loc[:, 'on_3b'] = data.on_3b.astype(int)
data.loc[:, 'on_1b'] = data.on_1b.astype(int)
data.loc[:, 'on_2b'] = data.on_2b.astype(int)

C:\Users\harry\AppData\Local\Temp\ipykernel_24512\4160384219.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[:, 'on_3b'] = data.on_3b.astype(int)
C:\Users\harry\AppData\Local\Temp\ipykernel_24512\4160384219.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[:, 'on_1b'] = data.on_1b.astype(int)
C:\Users\harry\AppData\Local\Temp\ipykernel_24512\4160384219.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[:, 'on_2b'] = data.on_2

In [3]:
dic = {'SL': 1, 'SI': 2, 'FF': 3, 'FS': 4, 'FC': 5, 'CU': 6, 'ST': 7}

In [85]:
X = data[['at_bat_number',
    'pitch_number',
    'same_hand_matchup',
    'balls', 'strikes',
    'outs', 'on_3b',
    'on_2b','on_1b',
    'inning', 'top_bot',
    'p_pitch_count',
    'pitcher_team',
]]
y = data['pitch_type'].map(dic)


In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)
# X_train = X[:1800]
# y_train = y[:1800]
# X_test = X[1801:]
# y_test = y[1801:]
X_test
y_test

2346    2
1771    3
1876    4
1574    3
793     2
       ..
1909    6
537     3
942     4
877     6
1978    3
Name: pitch_type, Length: 628, dtype: int64

In [160]:
class Network(nn.Module):
    def __init__(self, x):
        super().__init__()
        self.x = x
        self.layer1 = nn.Linear(self.x.shape[1], 100)
        self.layer2 = nn.Linear(100, 100)
        self.layer3 = nn.Linear(100, 7)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [161]:
model = Network(X_train)
model

Network(
  (layer1): Linear(in_features=13, out_features=100, bias=True)
  (layer2): Linear(in_features=100, out_features=100, bias=True)
  (layer3): Linear(in_features=100, out_features=7, bias=True)
)

In [175]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

X_train_tensor = torch.tensor(X_train.values).float()
y_train_tensor = torch.tensor(y_train.values).long()

for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()


In [176]:
model.eval()  # evaluation mode
X_test_tensor = torch.tensor(X_test.values).float()
y_test_tensor = torch.tensor(y_test.values).long()
with torch.no_grad():
    outputs = model(X_test_tensor)              # predictions (logits)
    preds = torch.argmax(outputs, dim=1) # class with highest score
    accuracy = (preds == y_test_tensor).float().mean()

print(f"Test Accuracy: {accuracy.item():.4f}")

Test Accuracy: 0.3822


In [173]:
model.eval()

row = X_test.iloc[-5]
row_tensor = torch.tensor(row.values).float().unsqueeze(0)

with torch.no_grad():
    logits = model(row_tensor)
    probs = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()

print("Raw logits:", logits)
print("Probabilities:", probs)
print("Predicted class:", predicted_class)

Raw logits: tensor([[-7.4962, -2.5312, -0.3496,  1.5212,  0.6018,  0.2503,  0.8034]])
Probabilities: tensor([[5.1862e-05, 7.4321e-03, 6.5848e-02, 4.2758e-01, 1.7051e-01, 1.1998e-01,
         2.0860e-01]])
Predicted class: 3
